### TORCH.AUTOGRAD를 사용한 자동 미분

In [1]:
import torch

x=torch.ones(5)
y=torch.zeros(3)
w=torch.randn(5,3,requires_grad=True)
b=torch.randn(3,requires_grad=True)
z=torch.matmul(x,w)+b
loss=torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [2]:
loss.backward()  # requires_grad=True로 설정된 모든 텐서들에 대해 그레디언트를 계산. backward() 붙히는 곳이 그레디언트 시작점
print(w.grad)   # loss를 w로 미분
print(b.grad)   # loss를 b로 미분

tensor([[0.3196, 0.0222, 0.0127],
        [0.3196, 0.0222, 0.0127],
        [0.3196, 0.0222, 0.0127],
        [0.3196, 0.0222, 0.0127],
        [0.3196, 0.0222, 0.0127]])
tensor([0.3196, 0.0222, 0.0127])


### 변화도 추적 멈추기

In [3]:
z = torch.matmul(x, w)+b
print(z.requires_grad)   # requires_grad=True: 모든연산 기록을 추적

with torch.no_grad():    # torch.no_grad(): 연산 추적 방지(메모리 사용 방지)
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [4]:
z = torch.matmul(x, w)+b
z_det = z.detach()   # detach(): 이것도 연산 추적 방지
print(z_det.requires_grad)

False


### 연산그래프에 대한 추가 정보

In [16]:
inp = torch.eye(5, requires_grad=True) # torc.eye(): 대각선이 1이고 나머지가 0인 행렬
print("inp:{}\n".format(inp))
out = (inp+1).pow(2)
print("out:{}\n".format(out))
out.backward(torch.ones_like(inp), retain_graph=True) # retain_graph=True: backward를 여러번 해도 다 계산해줌
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()  # backward는 여러번 할 수록 누적되기에 초기화 시키고 싶을 때 변화율을 0이로 만듬
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

inp:tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)

out:tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
